In [1]:
import requests
import json
import pandas as pd
import csv
import psycopg2

# Extraction Layer

In [2]:
url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

querystring = {"limit":"100000"}

headers = {
	"x-rapidapi-key": "b7d00ef2a3mshd1ecc6521d57cbap15f3dejsncfb5ad86adb7",
	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

#print(response.json())

data=response.json()

#Save data into a json file
filename="PropertyRecords.json"
with open (filename,'w') as file:
    json.dump (data,file,indent=4)

In [3]:
#Read into a dataframe
propertyrecord_df=pd.read_json("PropertyRecords.json")

In [4]:
propertyrecord_df.head()

,addressLine1,city,state,zipCode,formattedAddress,features,county,id,longitude,latitude,...,zoning,bathrooms,lotSize,propertyType,lastSaleDate,taxAssessment,propertyTaxes,owner,lastSalePrice,addressLine2
0,9277 Cranston Rd,Morehead,KY,40351,"9277 Cranston Rd, Morehead, KY 40351",{},Rowan,"9277-Cranston-Rd,-Morehead,-KY-40351",-83.394174,38.309714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,613 N Union St,Warsaw,IN,46580,"613 N Union St, Warsaw, IN 46580","{'floorCount': 2, 'foundationType': 'Raised', ...",Kosciusko,"613-N-Union-St,-Warsaw,-IN-46580",-85.862495,41.243530,...,R-3,2.0,8712.0,Single Family,2010-11-12T00:00:00.000Z,"{'2020': {'value': 99100, 'land': 12000, 'impr...","{'2019': {'total': 777}, '2022': {'total': 1266}}","{'names': ['ANTONIO RODRIGUEZ', 'ROSA RODRIGUE...",NaN,NaN
2,21579 Kings Bend Dr,Kingwood,TX,77339,"21579 Kings Bend Dr, Kingwood, TX 77339","{'architectureType': 'Conventional', 'cooling'...",Montgomery,"21579-Kings-Bend-Dr,-Kingwood,-TX-77339",-95.229829,30.063421,...,NaN,2.0,5249.0,Single Family,2020-11-13T00:00:00.000Z,"{'2022': {'value': 207310, 'land': 21990, 'imp...","{'2022': {'total': 5476}, '2023': {'total': 28...","{'names': ['Angel Morales', 'Sarah Abidin'], '...",NaN,NaN
3,4420 Northampton Dr,New Port Richey,FL,34653,"4420 Northampton Dr, New Port Richey, FL 34653","{'cooling': True, 'coolingType': 'Central', 'e...",Pasco,"4420-Northampton-Dr,-New-Port-Richey,-FL-34653",-82.682977,28.223399,...,MPUD,2.0,7750.0,Single Family,2023-08-03T00:00:00.000Z,"{'2021': {'value': 128250, 'land': 32564, 'imp...","{'2021': {'total': 1617}, '2022': {'total': 16...","{'names': ['JAMES H BULGER'], 'mailingAddress'...",370000.0,NaN
4,50 Aiken St,Norwalk,CT,6851,"50 Aiken St, Apt 491, Norwalk, CT 06851","{'exteriorType': 'Stone', 'fireplace': True, '...",Fairfield,"50-Aiken-St,-Apt-491,-Norwalk,-CT-06851",-73.421450,41.133960,...,C,1.0,NaN,Condo,1998-05-11T00:00:00.000Z,"{'2022': {'value': 144570, 'improvements': 144...","{'2022': {'total': 3553}, '2023': {'total': 36...","{'names': ['AMY J JONES'], 'mailingAddress': {...",92000.0,Apt 491


In [5]:
propertyrecord_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    int64  
 4   formattedAddress  500 non-null    object 
 5   features          445 non-null    object 
 6   county            500 non-null    object 
 7   id                500 non-null    object 
 8   longitude         500 non-null    float64
 9   latitude          500 non-null    float64
 10  assessorID        350 non-null    object 
 11  bedrooms          342 non-null    float64
 12  legalDescription  341 non-null    object 
 13  ownerOccupied     303 non-null    float64
 14  squareFootage     381 non-null    float64
 15  subdivision       286 non-null    object 
 16  yearBuilt         348 non-null    float64
 1

# Transformation Layer

In [6]:
# Convert dictionary column to string
propertyrecord_df['features']=propertyrecord_df['features'].apply(json.dumps)

In [7]:
# Replacing missing values
propertyrecord_df.fillna({
    'bedrooms':0,
    'squareFootage':0,
    'yearBuilt':0,
'yearBuilt':0,
'assessorID':'Unknown',
'ownerOccupied':0,
'bathrooms':0,
'lotSize':0,
'propertyType':'Unknown',
'taxAssessment':'Not Available',
'propertyTaxes':'Not Available',
'owner':'Unknown',
'addressLine2':'Not Available',
'legalDescription':'Not Available',
'subdivision':'Not Available',
'zoning':'Unknown',
'lastSalePrice':0,
'lastSaleDate':'Not Available',
'features':'None',
'county':'Not Available'},inplace=True)

In [8]:
propertyrecord_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    int64  
 4   formattedAddress  500 non-null    object 
 5   features          500 non-null    object 
 6   county            500 non-null    object 
 7   id                500 non-null    object 
 8   longitude         500 non-null    float64
 9   latitude          500 non-null    float64
 10  assessorID        500 non-null    object 
 11  bedrooms          500 non-null    float64
 12  legalDescription  500 non-null    object 
 13  ownerOccupied     500 non-null    float64
 14  squareFootage     500 non-null    float64
 15  subdivision       500 non-null    object 
 16  yearBuilt         500 non-null    float64
 1

In [9]:
# Create the fact table
fact_columns=['addressLine1','city','state','zipCode','formattedAddress','squareFootage','yearBuilt','bathrooms','bedrooms','lotSize','propertyType','longitude','latitude']
fact_table=propertyrecord_df[fact_columns]

In [10]:
fact_table.head()

,addressLine1,city,state,zipCode,formattedAddress,squareFootage,yearBuilt,bathrooms,bedrooms,lotSize,propertyType,longitude,latitude
0,9277 Cranston Rd,Morehead,KY,40351,"9277 Cranston Rd, Morehead, KY 40351",0.0,0.0,0.0,0.0,0.0,Unknown,-83.394174,38.309714
1,613 N Union St,Warsaw,IN,46580,"613 N Union St, Warsaw, IN 46580",1456.0,1915.0,2.0,4.0,8712.0,Single Family,-85.862495,41.243530
2,21579 Kings Bend Dr,Kingwood,TX,77339,"21579 Kings Bend Dr, Kingwood, TX 77339",1750.0,2006.0,2.0,3.0,5249.0,Single Family,-95.229829,30.063421
3,4420 Northampton Dr,New Port Richey,FL,34653,"4420 Northampton Dr, New Port Richey, FL 34653",1555.0,1989.0,2.0,3.0,7750.0,Single Family,-82.682977,28.223399
4,50 Aiken St,Norwalk,CT,6851,"50 Aiken St, Apt 491, Norwalk, CT 06851",736.0,1979.0,1.0,1.0,0.0,Condo,-73.421450,41.133960


In [11]:
# Creating location Dimension
location_dim=propertyrecord_df[['addressLine1','city','state','zipCode','longitude','latitude']].drop_duplicates().reset_index(drop=True)
location_dim.index.name='location_id'

In [12]:
location_dim.head()

,addressLine1,city,state,zipCode,longitude,latitude
location_id,,,,,,
0,9277 Cranston Rd,Morehead,KY,40351,-83.394174,38.309714
1,613 N Union St,Warsaw,IN,46580,-85.862495,41.243530
2,21579 Kings Bend Dr,Kingwood,TX,77339,-95.229829,30.063421
3,4420 Northampton Dr,New Port Richey,FL,34653,-82.682977,28.223399
4,50 Aiken St,Norwalk,CT,6851,-73.421450,41.133960


In [13]:
# Creating Sales Dimension
sales_dim=propertyrecord_df[['lastSalePrice','lastSaleDate']].drop_duplicates().reset_index(drop=True)
sales_dim.index.name='sales_id'

In [14]:
sales_dim.head()

,lastSalePrice,lastSaleDate
sales_id,,
0,0.0,Not Available
1,0.0,2010-11-12T00:00:00.000Z
2,0.0,2020-11-13T00:00:00.000Z
3,370000.0,2023-08-03T00:00:00.000Z
4,92000.0,1998-05-11T00:00:00.000Z


In [15]:
# Create property feature dimension
feature_dim=propertyrecord_df[['features','propertyType','zoning']].drop_duplicates().reset_index(drop=True)
feature_dim.index.name='feature_id'

In [16]:
feature_dim.head()

,features,propertyType,zoning
feature_id,,,
0,{},Unknown,Unknown
1,"{""floorCount"": 2, ""foundationType"": ""Raised"", ...",Single Family,R-3
2,"{""architectureType"": ""Conventional"", ""cooling""...",Single Family,Unknown
3,"{""cooling"": true, ""coolingType"": ""Central"", ""e...",Single Family,MPUD
4,"{""exteriorType"": ""Stone"", ""fireplace"": true, ""...",Condo,C


In [17]:
# Loading all into a csv file
fact_table.to_csv('property_fact.csv',index=False)
location_dim.to_csv('location_dimension.csv',index=True)
sales_dim.to_csv('sales_dimension.csv',index=True)
feature_dim.to_csv('features_dimension.csv',index=True)

# Loading layer

In [18]:
#develop a function to connect to pgadmin
def get_db_connection():
    connection=psycopg2.connect(
    host='localhost',
    database='postgres',
    port='5432',
    user='postgres',
    password='Nnekibe01'
)
    return connection

In [19]:
conn=get_db_connection()

In [20]:
# Creating database tables
def create_tables():
    conn=get_db_connection()
    cursor=conn.cursor()
    create_table_query='''
    CREATE SCHEMA IF NOT EXISTS zapbank;
    DROP TABLE IF EXISTS zipbank.fact_table;
    DROP TABLE IF EXISTS zipbank.location_dim;
    DROP TABLE IF EXISTS zipbank.sales_dim;
    DROP TABLE IF EXISTS zipbank.feature_dim;
    
    CREATE TABLE zapbank.fact_table(
    addressLine1 VARCHAR(255),
    city VARCHAR(100),
    state VARCHAR(50),
    zipcode INTEGER,
    formattedAddress VARCHAR(255),
    squareFootage FLOAT,
    yearBuilt FLOAT,
    bathrooms FLOAT,
    bedrooms FLOAT,
    lotSize FLOAT,
    propertyType VARCHAR(100),
    longitude FLOAT,
    latitude FLOAT
    );
    
    CREATE TABLE zapbank.location_dim(
    location_id SERIAL PRIMARY KEY,
    addressLine1 VARCHAR(255),
    city VARCHAR(100),
    state VARCHAR(50),
    zipcode INTEGER,
    longitude FLOAT,
    latitude FLOAT
    );
    
    CREATE TABLE zapbank.sales_dim(
    sales_id SERIAL PRIMARY KEY,
    lastSalePrice FLOAT,
    lastSaleDate DATE
    );
    
    CREATE TABLE zapbank.feature_dim(
    feature_id SERIAL PRIMARY KEY,
    features TEXT,
    propertyType VARCHAR(100),
    zoning VARCHAR(100)
    );
    '''
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()
    
create_tables()

In [21]:
# Create a function to load the csv data into the database
def load_data_from_csv_to_table(csv_path, table_name):
    conn=get_db_connection()
    cursor=conn.cursor()
    with open(csv_path,'r',encoding='utf-8') as file:
        reader=csv.reader(file)
        next(reader) #skip the header row
        for row in reader:
            placeholders=', '.join(['%s'] * len(row))
            query=f'INSERT INTO {table_name} VALUES ({placeholders})';
            cursor.execute(query,row)
        conn.commit()
        cursor.close()
        conn.close()
      

In [23]:
# loading fact table
fact_csv_path=r'C:\Users\USER\PostgresPipeline Project\property_fact.csv'
load_data_from_csv_to_table(fact_csv_path,'zapbank.fact_table')

In [24]:
# location dimension table
location_csv_path=r'C:\Users\USER\PostgresPipeline Project\location_dimension.csv'
load_data_from_csv_to_table(location_csv_path,'zapbank.location_dim')

In [25]:
#Loading features table
feature_csv_path=r'C:\Users\USER\PostgresPipeline Project\features_dimension.csv'
load_data_from_csv_to_table(feature_csv_path,'zapbank.feature_dim')

In [26]:
def load_data_from_csv_to_sales_table(csv_path,table_name):
    conn=get_db_connection()
    cursor=conn.cursor()
    with open(csv_path,'r',encoding='utf-8') as file:
        reader=csv.reader(file)
        next(reader) # to skip the header row
        for row in reader:
            #convert empty string(not available) in date column to none (null in sql)
            row=[None if (cell == '' or cell == 'Not Available') and col_name == 'lastSaleDate' else cell for cell,col_name in zip(row,sale_dim_column)]
            placeholders=', '.join(['%s'] * len(row))
            query=f'INSERT INTO {table_name} VALUES ({placeholders})';
            cursor.execute(query,row)
        conn.commit()
        cursor.close()
        conn.close()

#define the column name in sales_dim table
sale_dim_column=['sales_id','lastSalePrice','lastSaleDate']


In [27]:
# Sales dimension table
sales_csv_path=r'C:\Users\USER\PostgresPipeline Project\sales_dimension.csv'
load_data_from_csv_to_sales_table(sales_csv_path,'zapbank.sales_dim')

In [28]:
print ('All data has been loaded successfully into their respective schema and tables')

All data has been loaded successfully into their respective schema and tables
